In [67]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup

In [53]:


def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns true if the response seems to be HTML, false otherwise
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)



In [104]:
#cities to scrape
#Berlin, Hamburg, Munich, Cologne, Frankfurt am Main

#https://www.wetter.de/deutschland/wetter-berlin-18228265.html
#https://www.wetter.de/deutschland/wetter-hamburg-18219464.html
#https://www.wetter.de/deutschland/wetter-muenchen-18225562.html
#https://www.wetter.de/deutschland/wetter-koeln-18220679.html
#https://www.wetter.de/deutschland/wetter-frankfurt-18221009.html



raw_html = simple_get('https://www.wetter.de/deutschland/wetter-frankfurt-18221009/tag-9.html')
print(raw_html)

b'<!DOCTYPE html>\n\n<!--[if lt IE 7]>\n<html class="no-js lt-ie9 lt-ie8 lt-ie7" lang="de"> <![endif]-->\n<!--[if IE 7]>\n<html class="no-js lt-ie9 lt-ie8" lang="de"> <![endif]-->\n<!--[if IE 8]>\n<html class="no-js lt-ie9" lang="de"> <![endif]-->\n<!--[if gt IE 8]><!-->\n<html lang="de"> <!--<![endif]-->\n\n\n<head>\n\n    <!-- partials/site/html/head -->\n<script src="//cdn.optimizely.com/js/5078271027.js"></script>\n\n<!-- head title -->\n<title>Wetter Frankfurt, Deutschland - Wettervorhersage f\xc3\xbcr Frankfurt - wetter.de</title>\n\n<!-- head meta -->\n<meta name="msvalidate.01" content="6593CA878189EEEF3510F38B05DE90A1" >\n<meta name="description" content="Aktuelles Wetter f\xc3\xbcr Frankfurt. Bei Wetter.de finden Sie die Wettervorhersage f\xc3\xbcr heute und die n\xc3\xa4chsten 15 Tage im 1-Stunden-Takt. Alle Wetterdaten f\xc3\xbcr Frankfurt." >\n<meta name="keywords" content="Wetter, Wetterbericht, Wettervorschau, Wettervorhersage, Wetter Frankfurt, Wetterbericht Frankfurt, 

In [105]:
soup = BeautifulSoup(raw_html, 'html.parser')

In [108]:
temps=soup.find_all("span", class_="temperature")


In [109]:
temps

[<span class="temperature">13°</span>,
 <span class="temperature">12°</span>,
 <span class="temperature">12°</span>,
 <span class="temperature">12°</span>,
 <span class="temperature">11°</span>,
 <span class="temperature">11°</span>,
 <span class="temperature">11°</span>,
 <span class="temperature">12°</span>,
 <span class="temperature">13°</span>,
 <span class="temperature">15°</span>,
 <span class="temperature">16°</span>,
 <span class="temperature">17°</span>,
 <span class="temperature">18°</span>,
 <span class="temperature">19°</span>,
 <span class="temperature">20°</span>,
 <span class="temperature">20°</span>,
 <span class="temperature">21°</span>,
 <span class="temperature">21°</span>,
 <span class="temperature">20°</span>,
 <span class="temperature">20°</span>,
 <span class="temperature">18°</span>,
 <span class="temperature">17°</span>,
 <span class="temperature">16°</span>,
 <span class="temperature">15°</span>,
 <span class="temperature">14°</span>]

In [110]:
for i in temps:
    print(i.text)

13°
12°
12°
12°
11°
11°
11°
12°
13°
15°
16°
17°
18°
19°
20°
20°
21°
21°
20°
20°
18°
17°
16°
15°
14°


25